In [1]:
# import spacy
# from spacy import displacy

# nlp = spacy.load("pt_core_news_lg")
# ner = nlp.get_pipe('ner')

In [1]:
import pandas as pd
import requests
import sqlite3

from collections import defaultdict
from pathlib import Path
from zipfile import ZipFile

In [2]:
# ZipFile('camsap2.zip').extractall('model')

In [3]:
# nlp = spacy.load(Path('model'))

In [4]:
def get_data(year : int):
    url = 'https://sapl.santoantoniodaplatina.pr.leg.br/api/materia/materialegislativa'
    params = {
        'ano' : year,
        'tipo' : 13,
        'page' : None
    }
    pagination = {'next_page' : 1}
    data = list()

    while pagination['next_page'] != None:
      params['page'] = pagination['next_page']
      response = requests.get(url, params=params)
      content = response.json()
      pagination = content['pagination']
      results = content['results']
      for r in results:
        data.append({
            '__str__' : r['__str__'],
            'numero'  : r['numero'],
            'ementa'  : r['ementa'],
            'autores' : r['autores'],
            'id' : r['id'],
            'numero_protocolo' : r['numero_protocolo'],
            'texto_original' : r['texto_original']
        })
    
    return pd.DataFrame(data)

In [5]:
con = sqlite3.connect("requerimentos.db")

In [7]:
years = [2021, 2022, 2023]
data = list()
for year in years:
    df = get_data(year)
    df.insert(1, 'ano', year)
    df = df.rename(columns={'__str__' : 'requerimento'})
    data.append(df)
    

In [8]:
data[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   requerimento      690 non-null    object
 1   ano               690 non-null    int64 
 2   numero            690 non-null    int64 
 3   ementa            690 non-null    object
 4   autores           690 non-null    object
 5   id                690 non-null    int64 
 6   numero_protocolo  690 non-null    int64 
 7   texto_original    690 non-null    object
dtypes: int64(4), object(4)
memory usage: 43.2+ KB


In [9]:
data[0].head()

,requerimento,ano,numero,ementa,autores,id,numero_protocolo,texto_original
0,Requerimento de Vereador(a) nº 1 de 2021,2021,1,"Requerimento n° 01/2021, do Vereador Odemir Ja...",[13],2664,48,https://sapl.santoantoniodaplatina.pr.leg.br/m...
1,Requerimento de Vereador(a) nº 2 de 2021,2021,2,"Requerimento n° 02/2021, do Vereador Odemir Ja...",[13],2665,49,https://sapl.santoantoniodaplatina.pr.leg.br/m...
2,Requerimento de Vereador(a) nº 3 de 2021,2021,3,"Requerimento n° 03/2021, do Vereador Odemir Ja...",[13],2666,50,https://sapl.santoantoniodaplatina.pr.leg.br/m...
3,Requerimento de Vereador(a) nº 4 de 2021,2021,4,"Solicita ao Executivo que proceda a ""operação ...",[142],2670,62,https://sapl.santoantoniodaplatina.pr.leg.br/m...
4,Requerimento de Vereador(a) nº 5 de 2021,2021,5,Solicita ao Executivo pavimentação asfáltica n...,[14],2671,63,https://sapl.santoantoniodaplatina.pr.leg.br/m...


In [10]:
for df in data:
    df['autores'] = df['autores'].apply(lambda value : ','.join(str(x) for x in value))
    df.to_sql('requerimento', con, index=False, if_exists='append')

In [11]:
df = pd.concat(data, axis=0)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1677 entries, 0 to 298
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   requerimento      1677 non-null   object 
 1   ano               1677 non-null   int64  
 2   numero            1677 non-null   int64  
 3   ementa            1677 non-null   object 
 4   autores           1677 non-null   object 
 5   id                1677 non-null   int64  
 6   numero_protocolo  1675 non-null   float64
 7   texto_original    1671 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 117.9+ KB
